# Brainstorming:

Problem: 
- User has large pdf file that has factory test data. 
- PDF file is too large: 22k pages
- Data is in a table format
- Data inside table is messy, need parsing/cleaning

General solution:
- Use python to parse pdf file
- Get data from pdf file, parse data in desire format, export into excel file for visualization later

Specific solution, pipeline:
- Use camelot to parse tables from pdf files
- Detect desire table, concat them.
- Cleaning and organizing data into a nice/correct df:
  - Check and drop extra column (Done)
  - Detect and concat tables
  - Seperate first cell into extra columns
- Export df into excel file

Design note:
- Hardcoded: check all tables, if tables 7 columns: drop first one: all tables should have 6



# Functions

# Working code on small file:


In [11]:
import camelot
import pandas as pd
import numpy as np
from IPython.display import display
import time

def read_pdf(file_path):
    # Read the PDF file using Camelot
    print(f"Start reading PDF file {file_path}...")
    tables = camelot.read_pdf(file_path, pages='all' )
    # clean_tables(tables)
    return tables
def clean_table(table):
    print(f"\nStart cleaning table...")
    # Check if the table has more than 1 column
    if table.shape[1] > 1:
        print(f'Table has {table.shape[1]} columns. Cleaning up...')
        # # Drop the first column
        table = table.drop(columns=0)
        # Reset the column names
        table.columns = range(table.shape[1])
        print(f'Table now has {table.shape[1]} columns.')
    return table

def concatenate_split_tables(tables):
    # Record the start time
    start_time = time.time()

    print(f"\nStart concatenating split tables...")
    processed_tables = []

    # Initialize a variable to hold the current table
    current_table = None

    # Loop over all tables
    for table in tables:
        # Check the first line of the first row
        first_line = table.df.iloc[0, 0].split("\n")[0]
        print(f"First line: {first_line}")
        # If it starts a new table we're interested in
        if first_line.startswith("6.2.2 Maximum Output Power"):
            #print message found matched table at location:
            print(f"Found table '{first_line}' at page: {table.page}")
            # display(table.df)
            # If there's a current table, add it to the list of processed tables
            if current_table is not None:
                processed_tables.append(current_table)

            # Start a new current table
            current_table = table.df.copy()  # Create a deep copy

        # If it's a continuation of the previous table
        elif not first_line.startswith("6.2.3 Maximum Power Reduction"):
            print("Found continuation of previous table")
            # Clean the table
            cleaned_table = clean_table(table.df.copy())
            # Concatenate it with the current table
            current_table = pd.concat([current_table, cleaned_table])
            # print(f"Display concatenated table: {current_table}")
        else:
            print("Skip not matched table.")
    # Add the last table to the list of processed tables
    if current_table is not None:
        processed_tables.append(current_table)
    
    # Record and print the runtime
    end_time = time.time()
    runtime = end_time - start_time
    print(f"End concatenating split tables, took {runtime:.2f} seconds.")
    return processed_tables

def display_tables(tables):
    # Set the max column width to a high number (e.g., 1000) to display long contents
    pd.set_option('display.max_colwidth', 1000)

    # Display all tables
    print(f"Displaying {len(tables)} tables:")
    for i, table in enumerate(tables):
        table_number = i + 1
        print(f"\nTable {table_number}")
        display(table.df)

def display_processed_tables(tables):
    # Set the max column width to a high number (e.g., 1000) to display long contents
    pd.set_option('display.max_colwidth', 1000)

    # Display all processed tables
    print(f"\nDisplaying {len(tables)} tables:")
    for i, df in enumerate(tables):
        print(f"Table {i + 1}")
        display(df)
        

    
def clean_processed_tables(tables):
    print(f"Cleaning wrong data in measure and unit columns...")
    # Loop over all processed tables
    for i, table in enumerate(tables):
        # Loop over all rows in the table
        for j in table.index:
            # If "Unit" column contains a number and "dBm", split them
            if 'dBm' in str(table.loc[j, 'Unit']):
                measured, unit = str(table.loc[j, 'Unit']).split(' ')
                # print(f"measured: {measured}, unit: {unit}")
                # Move the number part to the "Measured" column
                table.loc[j, 'Measured'] = measured
                # Keep the "dBm" part in the "Unit" column
                table.loc[j, 'Unit'] = unit

        # Replace the table in the list with the cleaned table
        tables[i] = table

    return tables

def extract_band(tables):
    print(f"Extracting Band info...")
    for table in tables:
        line2 = table.iloc[0,0]
        if 'Band' in line2:
            band_info = line2.split(' ')
            for word in band_info:
                if word.startswith('Band'):
                    band_num = word[4:]  # Extract everything after "Band"
                    table.insert(1, 'Band', band_num)
                    break
            else:
                print(f"No 'Band' keyword in line: {line2}")
    return tables

def extract_info(tables):
    print(f"Extracting Testname, ULCH, BW, MOD, RD info...")
    for i, table in enumerate(tables):
        # Define patterns
        testname_pattern = r"(.*):@"
        ulch_pattern = r"ULCH: (\d+),"
        bw_pattern = r"BW: ([\d\.]+ MHz)"
        mod_pattern = r"UL_MOD_RB: ([^,]+),"
        rd_pattern = r"UL_MOD_RB: [^,]+, (.*)"

        # Extract info
        table['Testname'] = table.iloc[:,0].str.extract(testname_pattern)
        table['ULCH'] = table.iloc[:,0].str.extract(ulch_pattern)
        table['BW'] = table.iloc[:,0].str.extract(bw_pattern)
        table['MOD'] = table.iloc[:,0].str.extract(mod_pattern)
        table['RD'] = table.iloc[:,0].str.extract(rd_pattern)

        # Drop the first column
        table.drop(table.columns[0], axis=1, inplace=True)

        # Reorganize the columns
        print(f"Reorganizing columns...")
        new_column_order = ['Testname', 'Band', 'ULCH', 'BW', 'MOD', 'RD', 'Limit Low', 'Limit High', 'Measured', 'Unit', 'Status']
        table = table.reindex(columns=new_column_order)

        # Replace the table in the list with the cleaned and reorganized table
        tables[i] = table
        
    return tables      

def process_tables(tables):
    processed_tables = []
    
    for i, table in enumerate(tables):
        print(f"\nProcessing table {i+1}...")
        # Replace all-whitespace rows with NaN
        table = table.replace(r'^\s*$', np.nan, regex=True)
        # Drop the rows where all elements are NaN
        table = table.dropna(how='all')
        # print("Table after drop empty row:")
        # display(table)
    
        # Split the first cell of the first row and use it as column headers
        first_row = table.iloc[0, 0]
        
        headers = first_row.split('\n')  
        # print(f"First row: {first_row}")
        # print(f"Headers: {headers}")
        # headers.append("MissingHeader")
        table.columns = headers
        
        # Extract the table name from the first row
        table_name = table.iloc[0, 0].split('\n')[0]

        # Remove the first row
        table = table.iloc[1:]
        
        # Extract Band info
        print(f"\nExtracting Band info...")
        line2 = table.iloc[0,0]
        # print(f"line2: {line2}")
        if 'Band' in line2:
            band_info = line2.split(' ')
            for word in band_info:
                if word.startswith('Band'):
                    band_num = word[4:]  # Extract everything after "Band"
                    table.insert(1, 'Band', band_num)
                    break
            else:
                print(f"No 'Band' keyword in line: {line2}")
        # Drop the first row
        table = table.drop(table.index[0])
        # print("Table after header:")
        # display(table)
          
        # Check if the expected columns are in the table
        if 'Unit' not in table.columns:
            print(f"Table {i+1} doesn't have the expected structure. Skipping...")
            continue
  
        # Extract Testname, ULCH, BW, MOD, RD info
        print(f"\nExtracting Testname, ULCH, BW, MOD, RD info...")
        # Define patterns
        testname_pattern = r"(.*):@"
        ulch_pattern = r"ULCH: (\d+),"
        bw_pattern = r"BW: ([\d\.]+ MHz)"
        mod_pattern = r"UL_MOD_RB: ([^,]+),"
        rd_pattern = r"UL_MOD_RB: [^,]+, (.*)"

        # Extract info
        table['Testname'] = table.iloc[:,0].str.extract(testname_pattern)
        table['ULCH'] = table.iloc[:,0].str.extract(ulch_pattern)
        table['BW'] = table.iloc[:,0].str.extract(bw_pattern)
        table['MOD'] = table.iloc[:,0].str.extract(mod_pattern)
        table['RD'] = table.iloc[:,0].str.extract(rd_pattern)

        # Drop the first column
        table.drop(table.columns[0], axis=1, inplace=True)

        # Create a new column filled with the table name
        table.insert(0, 'Table Name', table_name)
        table = table.iloc[1:]
        
        # Reorganize the columns
        print(f"\nReorganizing columns...")
        new_column_order = ['Table Name', 'Testname', 'Band', 'ULCH', 'BW', 'MOD', 'RD', 'Limit Low', 'Limit High', 'Measured', 'Unit', 'Status']
        table = table.reindex(columns=new_column_order)

        # Reset the index
        table.reset_index(drop=True, inplace=True)
        
        #Append processed table to processed_tables:
        processed_tables.append(table)
        
        
    return processed_tables
    
def find_target_table(tables, desired_name):
    
    current_table = None
    desired_tables = []
    
    #Run through table_list
    for table in tables:
        first_cell = table.df.iloc[0,0].split('\n')[0]

        #If the cell in first row, first column has the desire format "6.x.x.x" AND the desire_name: Start a new table
        if first_cell.startswith("6.") and desired_name in first_cell:
            # Found start of new desired table
            print(f"Found start of new desired table: {first_cell}")
            if current_table is None:
                current_table = table
            current_table = table.df.copy()    
            
        elif first_cell.startswith("6.") and desired_name not in first_cell: 
            #else: return the current table, and reset the current_table to None
            print(f"\nSkipping a none-desired table: {first_cell}")
            if current_table is not None:
                # Save the current table
                desired_tables.append(current_table)
                #Reset, mark end of the desired table
                current_table = None
        
        #if the table doesnt match the desire_format AND there is a current_table: concat this table into the current table:         
        elif current_table is not None:
            # Continuation of previous desired table
            print(f"Found continuation of previous desired table: {table.df.iloc[0,0]}")
            # print(f"Table before clean: ")
            # display(table.df)
            #Clean table before concat:
            table.df = clean_table(table.df)
            current_table = pd.concat([current_table, table.df])

            # print(f"Table after concat:")
            # display(current_table)
        
            
    return desired_tables

long_file = "300_pages_extract.pdf"
short_file = "2_pages_test.pdf"

# #Read all tables from pdf
# short_tables = read_pdf(short_file)

# #Read all tables from pdf
# long_tables = read_pdf(long_file)


# Detect and concat the desire tables:

In [ ]:
def find_target_table(tables, desired_name):
    
    current_table = None
    desired_tables = []
    
    #Run through table_list
    for table in tables:
        first_cell = table.df.iloc[0,0].split('\n')[0]

        #If the cell in first row, first column has the desire format "6.x.x.x" AND the desire_name: Start a new table
        if first_cell.startswith("6.") and desired_name in first_cell:
            # Found start of new desired table
            print(f"Found start of new desired table: {first_cell}")
            if current_table is None:
                current_table = table
            current_table = table.df.copy()    
            
        elif first_cell.startswith("6.") and desired_name not in first_cell: 
            #else: return the current table, and reset the current_table to None
            print(f"\nSkipping a none-desired table: {first_cell}")
            if current_table is not None:
                # Save the current table
                desired_tables.append(current_table)
                #Reset, mark end of the desired table
                current_table = None
        
        #if the table doesnt match the desire_format AND there is a current_table: concat this table into the current table:         
        elif current_table is not None:
            # Continuation of previous desired table
            print(f"Found continuation of previous desired table: {table.df.iloc[0,0]}")
            # print(f"Table before clean: ")
            # display(table.df)
            #Clean table before concat:
            table.df = clean_table(table.df)
            current_table = pd.concat([current_table, table.df])

            # print(f"Table after concat:")
            # display(current_table)
        
            
    return desired_tables


#Read all tables from pdf
long_tables = camelot.read_pdf(long_file, pages='3-5')

desire_tables = find_target_table(long_tables, "6.2.2 Maximum Output Power")
# display_processed_tables(desire_tables)


clean_tables = process_tables(desire_tables)
display_processed_tables(clean_tables)

# Run on large file:

In [28]:
import re
from PyPDF2 import PdfReader

def find_table_pages(pdf_file, target_table):

    print(f"Searching for table: {target_table}")
    
    reader = PdfReader(pdf_file)

    pages_with_table = []

    for page_num in range(len(reader.pages)):

        print(f"Searching page {page_num} out of {len(reader.pages)}")
        
        page = reader.pages[page_num]
        text = page.extract_text()
        
        if target_table in text:
            print(f"Found table on page {page_num}")
            pages_with_table.append(page_num)

            # Check for multi-page
            if page_num < len(reader.pages)-2:

                # Get page_num+2 
                next_page = reader.pages[page_num+2]  
                next_text = next_page.extract_text()

                if re.search(r'\d+\.\d+\.\d+', next_text):
                    pages_with_table.append(page_num + 2)

            # Add extra page    
            if pages_with_table:
                pages_with_table.append(pages_with_table[-1] + 2)

    print(f"Table pages found: {pages_with_table}")
                
    return pages_with_table
def convert_to_ranges(numbers):
    """
    Convert a list of numbers into a list of ranges.
    """
    ranges = []
    for i in range(0, len(numbers), 2):
        ranges.append(f"{numbers[i]}-{numbers[i+1]}")
    return ranges


#  "25C_DATA_Extract(1temperatureOnly).pdf"
# "300_pages_extract.pdf"
long_file = "25C_DATA_Extract(1temperatureOnly).pdf"
target_table = "6.2.2 Maximum Output Power" 


# Find target table pages
pages = find_table_pages(long_file, target_table)

# Format page numbers for camelot
page_ranges = convert_to_ranges(pages)
print(f"Pages to read: {page_ranges}")



Searching for table: 6.2.2 Maximum Output Power
Searching page 0 out of 4436
Searching page 1 out of 4436
Searching page 2 out of 4436
Searching page 3 out of 4436
Found table on page 3
Searching page 4 out of 4436
Searching page 5 out of 4436
Searching page 6 out of 4436
Searching page 7 out of 4436
Searching page 8 out of 4436
Searching page 9 out of 4436
Searching page 10 out of 4436
Searching page 11 out of 4436
Searching page 12 out of 4436
Searching page 13 out of 4436
Searching page 14 out of 4436
Searching page 15 out of 4436
Searching page 16 out of 4436
Searching page 17 out of 4436
Searching page 18 out of 4436
Searching page 19 out of 4436
Searching page 20 out of 4436
Searching page 21 out of 4436
Searching page 22 out of 4436
Searching page 23 out of 4436
Searching page 24 out of 4436
Searching page 25 out of 4436
Searching page 26 out of 4436
Searching page 27 out of 4436
Searching page 28 out of 4436
Searching page 29 out of 4436
Searching page 30 out of 4436
Searching 

In [ ]:
clean_tables = []
   
for page_range in page_ranges:
    print(f"Processing page range: {page_range}")
 
    # Read all tables from pdf
    long_tables = camelot.read_pdf(long_file, pages=page_range)

    desire_tables = find_target_table(long_tables, "6.2.2 Maximum Output Power")
    # display_processed_tables(desire_tables)

    processed_tables = process_tables(desire_tables)
    #append to clean_tables:
    clean_tables.extend(processed_tables)


display_processed_tables(clean_tables)

In [31]:
# Concatenate all the dataframes in the list into a single dataframe
all_tables = pd.concat(clean_tables, ignore_index=True)
all_tables.head() # Display the first few rows of the resulting dataframe

In [33]:

# Export the dataframe to an Excel file
all_tables.to_excel("all_tables.xlsx", index=False)




,Table Name,Testname,Band,ULCH,BW,MOD,RD,Limit Low,Limit High,Measured,Unit,Status
0,6.2.2 Maximum Output Power,UE Maximum Output Power,1,18025,5.0 MHz,QPSK,8 (RB_Pos:LOW),20.30,25.70,NaN,22.69 dBm,Passed
1,6.2.2 Maximum Output Power,UE Maximum Output Power,1,18300,5.0 MHz,QPSK,1 (RB_Pos:LOW),20.30,25.70,NaN,22.63 dBm,Passed
2,6.2.2 Maximum Output Power,UE Maximum Output Power,1,18300,5.0 MHz,QPSK,8 (RB_Pos:LOW),20.30,25.70,NaN,22.63 dBm,Passed
3,6.2.2 Maximum Output Power,UE Maximum Output Power,1,18575,5.0 MHz,QPSK,1 (RB_Pos:HIGH),20.30,25.70,NaN,22.93 dBm,Passed
4,6.2.2 Maximum Output Power,UE Maximum Output Power,1,18575,5.0 MHz,QPSK,8 (RB_Pos:HIGH),20.30,25.70,NaN,22.97 dBm,Passed
